In [ ]:
using JLD2
using Statistics
using Printf
using Laplacians
using SparseArrays
using LinearAlgebra

## This notebook outputs the summary tables

In [ ]:
function combine(a, b)
    return [a; b]
end

function extractSlice(dic, slice)
    dicSlice = Dict()
    numepxr = length(dic["ne"])
    for (key, value) in dic
        if length(value) == numepxr
            dicSlice[key] = value[slice]
        else
            dicSlice[key] = value
        end
    end
    return dicSlice
end

In [ ]:
# SuiteSparse 
@load "suitesparse_all.jl.split02merge02.jld2"
dic_suitesparse = copy(dic);


In [ ]:
function hasZeroDiag(M::SparseMatrixCSC{Tv, Ti}) where {Tv, Ti}
    for i in 1:size(M,1)
        if M[i,i] == zero(Tv)
            return true
        end
    end
    return false
end

function hasZeroDiag(name::String)
    M = downloadSS(name)
    return hasZeroDiag(M)
end

In [ ]:
function extractSlice(dic, slice)
    dicSlice = Dict()
    numepxr = length(dic["ne"])
    for (key, value) in dic
        if length(value) == numepxr
            dicSlice[key] = value[slice]
        else
            dicSlice[key] = value
        end
    end
    return dicSlice
end

In [ ]:
sizeSlice = findall(dic_suitesparse["ne"] .> 1000)
dic_suitesparse = extractSlice(dic_suitesparse, sizeSlice);

In [ ]:
include("../julia-files/downloadSS.jl")
zeroDiagSlice = findall(hasZeroDiag, dic_suitesparse["testName"]);
noZeroDiagSlice = setdiff(1:length(sizeSlice), zeroDiagSlice);

In [ ]:
dic_suitesparse = extractSlice(dic_suitesparse, noZeroDiagSlice);

In [ ]:
# unweighted chimera
@load "uni_chimera_all.jl.split02merge02.n10000.h1.0.jld2"
dic_uniChimera = copy(dic);
@load "uni_chimera_all.jl.split02merge02.n100000.h2.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);
@load "uni_chimera_all.jl.split02merge02.n1000000.h4.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);
@load "uni_chimera_all.jl.split02merge02.n10000000.h8.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);

In [ ]:
# weighted chimera
@load "wted_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic_wtedChimera = copy(dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);

In [ ]:
# unweighted boundary chimera
@load "uni_bndry_chimera_all.jl.split02merge02.n10000.h1.0.jld2"
dic_uniBndryChimera = copy(dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n100000.h2.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n1000000.h4.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n10000000.h8.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);

In [ ]:
# weighted boundary chimera
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic_wtedBndryChimera = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);

In [ ]:
dic_chimera = copy(dic_uniChimera);
dic_chimera = merge(combine, dic_chimera, dic_wtedChimera);
dic_chimera = merge(combine, dic_chimera, dic_uniBndryChimera);
dic_chimera = merge(combine, dic_chimera, dic_wtedBndryChimera);

In [ ]:
# max flow 
@load "chimeraIPM_nopetsc.jl.split02merge02.jld2"
dic_maxFlow = copy(dic);
@load "spielmanIPM_all.jl.split02merge02.jld2"
dic_maxFlow = merge(combine, dic_maxFlow, dic);

In [ ]:
# Sachdeva star 
@load "sachdeva_star_all.jl.split02merge02.jld2"
dic_sachdevaStar = copy(dic);

In [ ]:
# SPE 
@load "spe_all.jl.split02merge02.jld2"
dic_spe = copy(dic);

In [ ]:
# uniform grid
@load "uniform_grid_all.jl.split02merge02.jld2"
dic_uniGrid = copy(dic);

In [ ]:
# checkered grid (high contrast grid)
@load "checkered_all.jl.split02merge02.jld2"
dic_checkeredGrid = copy(dic);

In [ ]:
# aniso grid 
@load "aniso_all.jl.split02merge02.jld2"
dic_anisoGrid = copy(dic);

In [ ]:
# weighted grid 
@load "wgrid_all.jl.split02merge02.jld2"
dic_wtedGrid = copy(dic);

In [ ]:
dic_grid = copy(dic_uniGrid);
dic_grid = merge(combine, dic_grid, dic_checkeredGrid);
dic_grid = merge(combine, dic_grid, dic_anisoGrid);
dic_grid = merge(combine, dic_grid, dic_wtedGrid);

### done

In [ ]:
# dicAll = [dic_suitesparse, dic_uniChimera, dic_wtedChimera, dic_uniBndryChimera, dic_wtedBndryChimera, dic_maxFlow, dic_sachdevaStar, dic_spe, dic_uniGrid, dic_checkeredGrid,dic_anisoGrid,dic_wtedGrid]
dicAll = [dic_suitesparse, dic_spe, dic_grid, dic_chimera, dic_sachdevaStar]

In [ ]:
# instanceNames = ["SuiteSparse Matrix Collection",  "Unweighted Chimeras", "Weighted Chimeras", "Unweighted SDDM Chimeras", "Weighted SDDM Chimeras", "Maximum flow IPMs", "Sachdeva stars", "SPE benchmark", "Uniform coefficient Poisson grid", "High contrast coefficient Poisson grid", "Anisotropic coef. Poisson grid, variable discretization", "Anisotropic coef. Poisson grid, variable weight"];
instanceNames = ["SuiteSparse Matrix Collection", "SPE benchmark", "Poisson grid problems", "Chimeras", "Sachdeva stars"]

In [ ]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.4g" sd(3,s)*1e6)

In [ ]:
solvers=["ac", "ac-s2m2", "cmg2", "hypre", "petsc_hypre",  "icc2"];

In [ ]:
function get_tvn_max(d, solvers)
    
    results_array = []
    for solver in solvers 
        if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
            cur_err = maximum(d[string(solver, "_err")])
            tvn = maximum((d[string(solver, "_tot")] ./ d["ne"]))
            if cur_err <= 1e-8
                push!(results_array, (solver, ffe6(tvn)))
            elseif cur_err < 1e-4
                push!(results_array, (solver, string(ffe6(tvn), "\\tnote{*}")))
            elseif cur_err < 1
                push!(results_array, (solver, string(ffe6(tvn), "\\tnote{**}")))
            else
                push!(results_array, (solver, "Inf"))
            end
        else
            push!(results_array, (solver, "N/A"))
        end
    end

    results = Dict(results_array)
    return results
end

In [ ]:
for (index, d) in enumerate(dicAll)
    max_tvns = get_tvn_max(d, solvers)
    line = string("{", instanceNames[index], "}&", join([max_tvns[solver] for solver in solvers], "&"), "\\\\")
    println(line)
end

## CMG problems

In [ ]:
cmg_failures = []
for (index, d) in enumerate(dicAll)
    for i in 1:length(d["testName"])
        if d["cmg_err"][i] == 1.0 || d["cmg_err"][i] == Inf
            push!(cmg_failures, d["testName"][i])
        end
    end
end


In [ ]:
cmg_failures

In [ ]:
cmg2_failures = []
for (index, d) in enumerate(dicAll)
    for i in 1:length(d["testName"])
        if d["cmg2_err"][i] == 1.0 || d["cmg2_err"][i] == Inf
            push!(cmg2_failures, d["testName"][i])
        end
    end
end

In [ ]:
cmg2_failures

## ICC Failure

In [ ]:
icc_failures = []
for (index, d) in enumerate(dicAll)
    for i in 1:length(d["testName"])
        if d["icc_err"][i] == 1.0 || d["icc_err"][i] == Inf
            push!(icc_failures, d["testName"][i])
        end
    end
end


In [ ]:
icc_failures

In [ ]:
icc2_failures = []
for (index, d) in enumerate(dicAll)
    for i in 1:length(d["testName"])
        # if d["icc2_err"][i] == 1.0 || d["icc2_err"][i] == Inf
        if d["icc2_err"][i] > 1e-8
            push!(icc2_failures, d["testName"][i])
        end
    end
end


In [ ]:
icc2_failures

In [ ]:
M = uni_bndry_chimera(100000,35);
b = randn(size(M,1))
b = M * b
b = b ./ norm(b);


In [ ]:
include("../julia-files/matlabSafe.jl")

In [ ]:
timeLimitIcc(Inf, M, b)

In [ ]:
for d in dicAll
    if maximum(d["icc2_err"]) > 1e-8
        println("Found a failure")
        @show d["testName"][argmax(d["icc2_err"])]
        @show maximum(d["icc2_err"])
    end
end

In [ ]:
a = uni_chimera(10000000,7)
c = components(a)
@show length(unique(c))

In [ ]:
include("../julia-files/downloadSS.jl")
include("../julia-files/isLap.jl")
include("../julia-files/compareSolvers.jl")
include("../julia-files/matlabSafe.jl")

In [ ]:
M = downloadSS("McRae/ecology1")
a, _ = adj(M)
c = components(a)
@show length(unique(c))

b = randn(size(M,1))
b = M * b
b = b ./ norm(b);

In [ ]:
isLap(M)

In [ ]:
# timeLimitIcc(Inf, M, b)
timeLimitIccLap(Inf, M, b)

In [ ]:
dic_test = Dict()

testLap([], dic_test, M, b; test_icc=true)